In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#method to determine the similarity string kernel for n-grams
def ngram_similarity(s1, s2, n = 4):
    nr = 0  # number of n-grams in common

    for i in range(len(s1)-n+1):
        ngram = s1[i:i+n]
        if ngram in s2:
            nr += 1

    return nr

# call the method to determine the similarity string kernel for n-grams
ngram_similarity('ananas copt', 'banana verde', 4)


2

In [4]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier


# Define the training data as a list of text documents
train_data = np.load ('train_data.npy')
train_labels = np.load ('train_labels.npy')

# transform the data into numpy arrays
train_data = np.array(train_data)
train_labels = np.array(train_labels)


train_mock_data = train_data [:800]
train_mock_labels = train_labels [:800]


validation_mock_data = train_data [800:]
validation_mock_labels = train_labels [800:]


class KNNClassifier:
    def __init__ (self, train_data, train_labels, k):
        self.train_data = train_data
        self.train_labels = train_labels
        self.k = k

    def predict_data (self, test_data):
       
       distances = [ngram_similarity(train_d, test_data, 8) for train_d in self.train_data]
       sorted_distances = np.argsort(distances)[::-1]
       nearest_index = sorted_distances[:self.k]
       k_nearest_labels = self.train_labels [nearest_index]


       return np.sign(np.sum(k_nearest_labels))
    

    def classify_data (self, test_data):
       predicted_labels = np.zeros(len(test_data))
       for i in range(len(test_data)):
              predicted_labels[i] = self.predict_data(test_data[i])

       return predicted_labels


    def accuracy (self, test_labels, predicted_labels):
        return np.mean(test_labels == predicted_labels)  


for k in [1, 3, 5, 7, 9, 11, 13, 15]: 
    
    knn = KNNClassifier(train_mock_data, train_mock_labels, k)

    predicted_labels = knn.classify_data(validation_mock_data)

    accuracy = knn.accuracy(validation_mock_labels, predicted_labels)

    print(accuracy)

# maximul e k = 5

# test data and print in a file the predicted labels
# test_data = np.load ('test_data.npy')
# test_data = np.array(test_data)

# with open('test_labels.txt', 'w') as f:
#     for i in range(len(test_data)):
#         f.write(str(knn.predict_data(test_data[i])) + '\n')

       

0.875
0.895
0.905
0.905
0.89
0.895
0.89
0.9


In [7]:
test_data = np.load ('test_data.npy')

def kernel_matrix_train(train_mock_data, n):
    # initialize the kernel matrix
    kernel = np.zeros((len(train_mock_data), len(train_mock_data)))

    # compute the similarity for each pair of strings
    for i in range(len(train_mock_data)):
        for j in range(len(train_mock_data)):
            kernel[i][j] = ngram_similarity(train_mock_data[i], train_mock_data[j], n)
    return kernel

kernel_train = kernel_matrix_train(train_mock_data, 8)
print (kernel_train)

# create kernel matrix for the train set and test set
def kernel_matrix_test(validation_mock_data,train_mock_data,  n):
    # initialize the kernel matrix
    kernel = np.zeros((len(validation_mock_data),len(train_mock_data)))

    # compute the similarity for each pair of strings
    for i in range(len(validation_mock_data)):
        for j in range(len(train_mock_data)):
            kernel[i][j] = ngram_similarity(validation_mock_data[i], train_mock_data[j], n)
    return kernel

# compute the kernel matrix for the train set and data set
kernel_test = kernel_matrix_test(validation_mock_data, train_mock_data, 8)
print (kernel_test)

[[136.   7.   0. ...   0.   0.   0.]
 [  7. 172.   0. ...  10.   0.   6.]
 [  0.   0.  47. ...   0.   0.   4.]
 ...
 [  0.   4.   0. ... 107.   0.   2.]
 [  0.   0.   0. ...   0. 164.   0.]
 [  0.   2.   4. ...   2.   0.  92.]]
[[ 0.  1.  0. ...  0.  0.  2.]
 [ 0.  1.  0. ...  1.  0.  3.]
 [ 0.  0.  0. ...  1.  9.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 9. 10. 14. ...  4.  0. 16.]
 [ 0.  0.  5. ...  1.  3.  9.]]


In [8]:
# train krr model with kernel precomputed
from sklearn.kernel_ridge import KernelRidge

# initialize the model
for alfa_val in [0.1, 1, 10, 100, 1000]:

    krr_model = KernelRidge(alpha=alfa_val, kernel='precomputed')
    
    krr_model.fit(kernel_train, train_mock_labels)

    # predict the labels for the test set
    predicted_labels = krr_model.predict(kernel_test)

    predictii = []

    for label in predicted_labels:
        if label <= 0 :
            predictii.append(-1)
        else :
            predictii.append(1)

    print(f"valoarea de predicti pt alpha = {alfa_val} este {(np.array(predictii) == validation_mock_labels).mean()}")

    # determine the accuracy of the model
    
    
    

   

valoarea de predicti pt alpha = 0.1 este 0.93
valoarea de predicti pt alpha = 1 este 0.93
valoarea de predicti pt alpha = 10 este 0.94
valoarea de predicti pt alpha = 100 este 0.95
valoarea de predicti pt alpha = 1000 este 0.91
